<a href="https://colab.research.google.com/github/ramatudataanalyst-ai/Ramatudataanalyst.github.io/blob/main/Customer_Churn_Revenue_Risk_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Churn & Revenue Risk Analysis

## Business Context
Customer churn is a critical problem for telecommunications companies because acquiring
new customers is significantly more expensive than retaining existing ones. High churn
directly impacts revenue, profitability, and long-term growth.

## Objective
The objective of this project is to analyze customer churn behavior, identify high-risk
customer segments, quantify potential revenue loss, and provide actionable,
data-driven recommendations to improve customer retention.

## Stakeholders
- Executive Management
- Customer Retention Team
- Marketing Department
- Finance Department

## Tools & Technologies
- Python (Google Colab)
- pandas, numpy
- matplotlib, seaborn
- SQL (SQLite)
- GitHub for version control


In [8]:
import sys
print(sys.version)


3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3


In [10]:
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

plt.rcParams["figure.figsize"] = (10, 6)
sns.set_theme(style="whitegrid")


## Dataset Description

The dataset used in this project is the **Telco Customer Churn Dataset**, which contains
customer demographic information, subscription details, billing data, and churn status.

### Key Fields
- Customer demographics (gender, senior citizen)
- Contract type and payment method
- Monthly and total charges
- Service subscriptions
- Churn indicator (Yes / No)


In [14]:
# Upload dataset manually in Colab
from google.colab import files

uploaded = files.upload()


Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn (1).csv


WA_Fn-UseC_-Telco-Customer-Churn.csv


In [17]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Validation & Quality Checks

Before analysis, the dataset is validated to ensure:
- Correct data types
- No unexpected missing values
- Logical consistency in revenue fields


In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [19]:
# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Remove rows with missing values
df = df.dropna()

df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


## Key Business Metrics

The following metrics are used to evaluate churn impact:
- Churn Rate
- Average Revenue Per User (ARPU)
- Revenue at Risk


In [20]:
churn_rate = df["Churn"].value_counts(normalize=True) * 100
churn_rate


,proportion
Churn,
No,73.42
Yes,26.58


In [21]:
revenue_at_risk = df[df["Churn"] == "Yes"]["MonthlyCharges"].sum()
revenue_at_risk


np.float64(139130.85)

## Churn Analysis by Contract Type

Understanding churn by contract type helps identify which customer agreements
are most vulnerable and require retention strategies.


In [22]:
df.groupby("Contract")["Churn"].value_counts(normalize=True).unstack() * 100


Churn,No,Yes
Contract,,
Month-to-month,57.29,42.71
One year,88.72,11.28
Two year,97.15,2.85


## SQL-Based Business Analysis

To simulate a real production environment, SQL is used to answer business
questions directly from a relational database.


In [23]:
conn = sqlite3.connect("churn.db")
df.to_sql("customers", conn, if_exists="replace", index=False)


7032

In [25]:
query = """
SELECT Contract,
       COUNT(*) AS customers,
       SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM customers
GROUP BY Contract
"""
pd.read_sql(query, conn)


,Contract,customers,churn_rate
0,Month-to-month,3875,42.71
1,One year,1472,11.28
2,Two year,1685,2.85


from matplotlib import pyplot as plt
_df_23['customers'].plot(kind='hist', bins=20, title='customers')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_24['churn_rate'].plot(kind='hist', bins=20, title='churn_rate')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_25.groupby('Contract').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_26.plot(kind='scatter', x='customers', y='churn_rate', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_28['churn_rate'].plot(kind='line', figsize=(8, 4), title='churn_rate')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_29['Contract'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_29, x='customers', y='Contract', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_30['Contract'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_30, x='churn_rate', y='Contract', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## Key Insights

- Month-to-month contracts exhibit the highest churn rate
- Long-term contracts significantly reduce churn
- Churned customers represent substantial recurring revenue loss


## Business Recommendations

- Incentivize month-to-month customers to migrate to annual contracts
- Focus retention campaigns on high-revenue, high-risk segments
- Bundle services to increase customer stickiness
- Prioritize proactive outreach before contract renewal
